# Running MAGICC in Parallel

The code in this notebook is a work in progress so it is quite verbose. In future prettier wrappers can be written but for now it's helpful to have things in one place.

In [1]:
import glob
import logging
import multiprocessing
import os.path
from concurrent.futures import ProcessPoolExecutor
from subprocess import CalledProcessError

import f90nml
import matplotlib.pyplot as plt
import numpy as np
from openscm_runner.adapters.magicc7._parallel_process import _parallel_process
from scmdata import df_append

from _magicc_instances import _MagiccInstances
from tqdm.autonotebook import tqdm

from matplotlib.lines import Line2D
import seaborn as sns



logger = logging.getLogger()
logger.setLevel(logging.INFO)

stderr_info_handler = logging.StreamHandler()
formatter = logging.Formatter("%(name)s - %(levelname)s: %(message)s")
stderr_info_handler.setFormatter(formatter)
logger.addHandler(stderr_info_handler)

<IPython.core.display.Javascript object>

/Users/susannebaur/anaconda3/lib/python3.7/site-packages/openscm_runner/run.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
File doesn't exist 


## Config

In [ ]:
# how many MAGICC workers to use
NWORKERS = 4

# where should MAGICC copies be made
MAGICC_ROOT_DIR = os.path.expanduser(os.path.join(
    ""))
MAGICC_ROOT_DIR

data_path = ''
plots_path = ''

# where is the MAGICC executable to copy
os.environ["MAGICC_EXECUTABLE_6"] = os.path.expanduser(os.path.join())

os.environ["MAGICC_EXECUTABLE_6"]


## Parallel setup

In [3]:
shared_manager = multiprocessing.Manager()
shared_dict = shared_manager.dict()

if not os.path.isdir(MAGICC_ROOT_DIR):
    os.makedirs(MAGICC_ROOT_DIR)

def init_magicc_worker(dict_shared_instances, root_dir):
    logger.debug("Initialising process %s", multiprocessing.current_process())
    logger.debug("Existing instances %s", dict_shared_instances)

def _run_func(magicc, cfg):
    try:
        scenario = cfg.pop("scenario")
        res = magicc.run(**cfg)
        res.set_meta(cfg["run_id"], "run_id")
        res.set_meta(scenario, "scenario")
        
        return res
    except CalledProcessError as e:
        # Swallow the exception, but return None
        logger.debug("magicc run failed: {} (cfg: {})".format(e.stderr, cfg))
        return None

instances = _MagiccInstances(existing_instances=shared_dict)




def _execute_run(cfg, run_func, setup_func):
    magicc = instances.get(root_dir=MAGICC_ROOT_DIR, init_callback=setup_func)

    return run_func(magicc, cfg)

def make_runs_list(cfgs):
    """
    Turn the configs into a list which can be run in parallel.
    
    Assigns ``run_id`` for each run if it's not already there.
    """
    out = [
        {
            "cfg": {**{"run_id": i}, **cfg},
            "run_func": _run_func,
            "setup_func": _setup_func,
        }
        for i, cfg in enumerate(cfgs)
    ]
    if not all(["scenario" in c["cfg"] for c in out]):
       raise KeyError("Please include a key 'scenario' in each config")
        
    return out

## Modify general MAGICC setup

In [4]:
def _setup_func(magicc):
    logger.info(
        "Setting up MAGICC worker in %s", magicc.root_dir,
    )


    magicc.set_config(
        # can set config to be used in all runs here e.g.
        # out_forcing=1
       
       #  OUT_CARBONCYCLE = 1, 
       # OUT_FORCING = 1,
        RF_TOTAL_CONSTANTAFTERYR =  2500,
  RF_TROPOZ_CONSTANTAFTERYR =  2500,
  RF_STRATOZ_CONSTANTAFTERYR =  2500,
    #    FILE_TUNINGMODEL_2 =  '', #C4MIP_UVIC

    )
    
        
    magicc.set_years(
        # modify start- and endyear
        endyear=2500  
    )

## Runs

First we need to get all our configs as a list of dictionaries, like the below.

In [5]:
# fetch the 600 probabilistic parameter sets from the MAGICC run directory
rundir=""
rundir_files = os.listdir(rundir)
probabilistic_files = [x for x in rundir_files if "MAGTUNE_DRAWNSET_CDF_RogeljIPCCrepresent_" in x ]

In [6]:
# one could also load the configs from the probabilistic sets using f90nml

# choose scenario
scenarios = [""] 

# load probabilistic sets
cfgs = []
for scen in scenarios:
    for f in probabilistic_files:
        nml = f90nml.read(rundir+f)["nml_allcfgs"]
        # add scenario information
        nml["file_emissionscenario"]=scen
        nml["scenario"]=scen.replace(".SCEN", "")        
        # append
        cfgs.append(nml)
           
#cfgs = [
#    {
#        "core_climatesensitivity": cs,
#        "rf_cloud_albedo_aer_wm2": rfcloud,
#        "file_emissionscenario": scen,
#        "scenario": scen.replace(".SCEN", "")
#    }
#    for cs, rfcloud in zip(
#        np.round(np.linspace(2, 6, 50), 2), 
#        np.round(np.linspace(-0.2, -1.5, 50), 2)
#    )
#    for scen in ["RCP26.SCEN", "RCP45.SCEN"]
#]

#cfgs[:1]

In [7]:
runs = make_runs_list(cfgs)
#runs[:1]

In [ ]:
try:
    pool = ProcessPoolExecutor(
        max_workers=NWORKERS,
        initializer=init_magicc_worker,
        initargs=(shared_dict, MAGICC_ROOT_DIR),
    )

    res_raw = _parallel_process(
        func=_execute_run,
        configuration=runs,
        pool=pool,
        config_are_kwargs=True,
        front_serial=2,
        front_parallel=2,
    )

    res = df_append([r for r in res_raw if r is not None])

finally:
    instances.cleanup()
    shared_manager.shutdown()
    pool.shutdown()

In [13]:
temp_world = res.filter(variable="Surface Temperature", region = 'World').process_over("run_id", "median").T
rf_world = res.filter(variable="Radiative Forcing", region = 'World').process_over("run_id", "median").T
em_world = res.filter(variable="KYOTOGHGS_GWPEMIS", region = 'World').process_over("run_id", "median").T

temp_world_17 = res.filter(variable="Surface Temperature", region = 'World').process_over("run_id", operation="quantile", q=0.17).T
temp_world_83 = res.filter(variable="Surface Temperature", region = 'World').process_over("run_id", operation="quantile", q=0.83).T
rf_world_17 = res.filter(variable="Radiative Forcing", region = 'World').process_over("run_id", operation="quantile", q=0.17).T
rf_world_83 = res.filter(variable="Radiative Forcing", region = 'World').process_over("run_id", operation="quantile", q=0.83).T



In [14]:

temp_world.to_csv(data_path + 'median_temperatures_csv/' + 'temp_med_NDC5_SCa.csv')
temp_world_17.to_csv(data_path + 'quantile_temperatures_csv/' + 'temp_q17_NDC5_SCa.csv')
temp_world_83.to_csv(data_path + 'quantile_temperatures_csv/' + 'temp_q83_NDC5_SCa.csv')

rf_world.to_csv(data_path + 'median_rf_csv/' + 'rf_med_NDC5_SCa.csv')
rf_world_17.to_csv(data_path + 'quantile_rf_csv/' + 'rf_q17_NDC5_SCa.csv')
rf_world_83.to_csv(data_path + 'quantile_rf_csv/' + 'rf_q83_NDC5_SCa.csv')

em_world.to_csv(data_path + 'em_world.csv')